In [1]:
import sys # TODO - remove sys.path.append once we have testpypi
sys.path.append('../../ablm/')
sys.path.append('../../ablm/abmap/') # torch.load requires the original module to be available. Adding to path for now

import torch
import abmap

/Users/I0479386/opt/anaconda3/envs/abmap/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load foundation models (default beplerberger)
abmap.reload_models_to_device() # Now allows for cpu, callable from root

cpu


In [3]:
# Load AbMAP --> make this into one function?
chkpt_bb_VH = torch.load('../checkpoints/abnet_beplerberger_H.pt', map_location=torch.device('cpu'))
chkpt_bb_VL = torch.load('../checkpoints/abnet_beplerberger_L.pt', map_location=torch.device('cpu'))
# TODO - save these configs somewhere?

abmap_H = abmap.AbMAPAttn(embed_dim=2200, mid_dim2=1024, mid_dim3=512,
                                      proj_dim=252, num_enc_layers=1, num_heads=16)
abmap_L = abmap.AbMAPAttn(embed_dim=2200, mid_dim2=1024, mid_dim3=512,
                                      proj_dim=252, num_enc_layers=1, num_heads=16)
abmap_H.load_state_dict(chkpt_bb_VH['model_state_dict'])
abmap_L.load_state_dict(chkpt_bb_VL['model_state_dict'])

<All keys matched successfully>

In [4]:
demo_seq = 'EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS'
x = abmap.ProteinEmbedding(demo_seq, chain_type='H', embed_device=torch.device('cpu'))
x.create_cdr_specific_embedding(embed_type='beplerberger', k=50)

data/anarci_files/temp0
Sequence: EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS
scheme: chothia


tensor([[ 0.4400, -0.0200,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        [-0.0200,  0.0000, -0.0200,  ...,  0.0000,  0.0000,  1.0000],
        [-0.0200, -0.0600,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        ...,
        [-0.0200, -0.0400, -0.0600,  ...,  0.0000,  1.0000,  1.0000],
        [-0.0200, -0.0400,  0.0000,  ...,  0.0000,  1.0000,  1.0000],
        [-0.0600, -0.0200, -0.0400,  ...,  0.0000,  1.0000,  1.0000]])

In [17]:
with torch.no_grad():
    embed_var = abmap_H.embed(x.cdr_embedding.unsqueeze(0))
    embed_fl = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='fixed')

In [18]:
embed_var.shape, embed_fl.shape

(torch.Size([1, 31, 256]), torch.Size([1, 512]))

## Profiling

In [19]:
import cProfile
import pstats

In [20]:
with cProfile.Profile() as profile:
    x = abmap.ProteinEmbedding(demo_seq, chain_type='H', embed_device=torch.device('cpu'))
    inp = x.create_cdr_specific_embedding(embed_type='beplerberger', k=20).unsqueeze(0)
    embed_fl = abmap_H.embed(inp, embed_type='fixed')
    ps = pstats.Stats(profile)
    ps.sort_stats('tottime', 'cumtime')
    ps.print_stats()

data/anarci_files/temp0
Sequence: EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS
scheme: chothia


         36666 function calls (36320 primitive calls) in 10.845 seconds

   Ordered by: internal time, cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       63   10.051    0.160   10.051    0.160 {built-in method torch.lstm}
        1    0.661    0.661    0.661    0.661 {built-in method posix.system}
       24    0.024    0.001    0.024    0.001 {built-in method torch.cat}
        1    0.020    0.020    0.024    0.024 /Users/I0479386/projects/ablm/examples/../../ablm/abmap/abmap_embed.py:115(create_cdr_embedding)
        2    0.020    0.010    0.020    0.010 {built-in method torch.stack}
        1    0.007    0.007   10.841   10.841 /Users/I0479386/projects/ablm/examples/../../ablm/abmap/abmap_embed.py:192(create_cdr_specific_embedding)
        1    0.005    0.005    9.309    9.309 /Users/I0479386/projects/ablm/examples/../../ablm/abmap/abmap_embed.py:78(create_kmut_matrix)
       20    0.005    0.000    0.011    0.001 /Users/I0479386/projects/